In [86]:
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px


In [63]:
links = pd.read_parquet("data_clean/links.parquet")
vessels = pd.read_parquet("data_clean/vessels.parquet")
locations = pd.read_parquet("data_clean/locations.parquet")
deliveries = pd.read_parquet("data_clean/deliveries.parquet")
fish = pd.read_parquet("data_clean/fish.parquet")

trans = pd.read_parquet("data_clean/transactions.parquet")
ping = pd.read_parquet("data_clean/pings.parquet")
harbor = pd.read_parquet("data_clean/harbors.parquet")

In [64]:
# TODO: transform fish species present into list of categories
locations["fish_species_list"] = (
    locations["fish_species_present"]
    .fillna("")
    .str.replace(r"\s+", "", regex=True)        
    .str.split(",")                             
)

In [65]:
sea_area = locations.loc[locations["kind"].isin(["Fishing Ground", "Ecological Preserve"])]
sea_area

,location_id,Activities,fish_species_present,kind,fish_species_list
18,Cod Table,Commercial fishing,"Cod/Gadus n.specificatae, Birdseye/Pisces frig...",Fishing Ground,"[Cod/Gadusn.specificatae, Birdseye/Piscesfrigu..."
19,Ghoti Preserve,"Research, Tourism, Recreation","Wrasse/Labridae n.refert, Beauvoir/Habeas pisc...",Ecological Preserve,"[Wrasse/Labridaen.refert, Beauvoir/Habeaspisce..."
20,Wrasse Beds,Commercial fishing,"Wrasse/Labridae n.refert, Birdseye/Pisces frig...",Fishing Ground,"[Wrasse/Labridaen.refert, Birdseye/Piscesfrigu..."
21,Nemo Reef,"Recreation, Tourism","Wrasse/Labridae n.refert, Tuna/Thunnini n.vera...",Ecological Preserve,"[Wrasse/Labridaen.refert, Tuna/Thunninin.vera,..."
22,Don Limpet Preserve,"Recreation, Tourism","Tuna/Thunnini n.vera, Birdseye/Pisces frigus, ...",Ecological Preserve,"[Tuna/Thunninin.vera, Birdseye/Piscesfrigus, B..."
23,Tuna Shelf,"Commercial fishing, Sport fishing","Tuna/Thunnini n.vera, Birdseye/Pisces frigus, ...",Fishing Ground,"[Tuna/Thunninin.vera, Birdseye/Piscesfrigus, B..."


In [66]:
cities = locations.loc[locations["kind"].isin(["city"])]
cities

,location_id,Activities,fish_species_present,kind,fish_species_list
0,City of Haacklee,"Tourism, Local shipping",Unknown,city,[Unknown]
1,City of Lomark,"Deep sea fishing, Commercial fishing, Tourism,...",Unknown,city,[Unknown]
2,City of Himark,"Recreation, tourism",Unknown,city,[Unknown]
3,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,[Unknown]
4,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,[Unknown]
5,City of Port Grove,"Tourism, Research",Unknown,city,[Unknown]


In [67]:
import re

fish_clean = fish.copy()
sea_clean = sea_area.copy()

#  Normalizza i nomi dei pesci
def normalize_name(x):
    if pd.isna(x):
        return ''
    x = x.lower()
    x = re.sub(r'[^a-z/]', '', x)   # rimuove tutto tranne lettere e '/'
    return x.strip()

fish_clean['fish_norm'] = fish_clean['entity_name'].apply(normalize_name)

# Esplodi fish_species_list se è lista, altrimenti converti in lista
if isinstance(sea_clean['fish_species_list'].iloc[0], list):
    sea_exp = sea_clean.explode('fish_species_list')
else:
    sea_exp = sea_clean.assign(fish_species_list=sea_clean['fish_species_list'].str.split(',')).explode('fish_species_list')

sea_exp['fish_species_list'] = sea_exp['fish_species_list'].astype(str).str.strip()
sea_exp['fish_norm'] = sea_exp['fish_species_list'].apply(normalize_name)

# Merge sulle versioni normalizzate
fish_locations = fish_clean.merge(
    sea_exp,
    on='fish_norm',
    how='inner'
)[['entity_name', 'location_id', 'kind']].drop_duplicates()

print(fish_locations)


                 entity_name          location_id                 kind
0   Cod/Gadus n.specificatae            Cod Table       Fishing Ground
1     Birdseye/Pisces frigus            Cod Table       Fishing Ground
2     Birdseye/Pisces frigus          Wrasse Beds       Fishing Ground
3     Birdseye/Pisces frigus            Nemo Reef  Ecological Preserve
4     Birdseye/Pisces frigus  Don Limpet Preserve  Ecological Preserve
5     Birdseye/Pisces frigus           Tuna Shelf       Fishing Ground
6    Sockfish/Pisces foetida  Don Limpet Preserve  Ecological Preserve
7   Wrasse/Labridae n.refert       Ghoti Preserve  Ecological Preserve
8   Wrasse/Labridae n.refert          Wrasse Beds       Fishing Ground
9   Wrasse/Labridae n.refert            Nemo Reef  Ecological Preserve
10    Beauvoir/Habeas pisces            Cod Table       Fishing Ground
11    Beauvoir/Habeas pisces       Ghoti Preserve  Ecological Preserve
12    Beauvoir/Habeas pisces          Wrasse Beds       Fishing Ground
13    

In [68]:
# PER CAPIRE QUALI PESCI NON POSSONO ESSERE PESCATI

links = (
    fish_locations.assign(
        location_label=lambda df: df['location_id'].astype(str) + " (" + df['kind'] + ")"
    )
    .groupby(['entity_name', 'location_label'])
    .size()
    .reset_index(name='value')
)

nodes = pd.Index(pd.concat([links['entity_name'], links['location_label']]).unique())

links['source_id'] = nodes.get_indexer(links['entity_name'])
links['target_id'] = nodes.get_indexer(links['location_label'])

fig = go.Figure(data=[go.Sankey(
    arrangement="snap",
    node=dict(
        label=nodes.tolist(),
        pad=20,
        thickness=18,
        line=dict(color="black", width=0.5)
    ),
    link=dict(
        source=links['source_id'],
        target=links['target_id'],
        value=links['value']
    )
)])

fig.update_layout(
    title_text="Fish → Location",
    font_size=10,
    height=600
)

fig.show()


In [69]:
# --- 1️⃣ Seleziona il porto/città di destinazione ---
available_cities = cities['location_id'].unique()
print("Porti/Città disponibili:", available_cities)

Porti/Città disponibili: ['City of Haacklee' 'City of Lomark' 'City of Himark' 'City of Paackland'
 'City of South Paackland' 'City of Port Grove']


In [70]:
deliveries

,delivery_cargo_id,date,qty_tons
0,cargo_2035_2394778c,2035-11-03,24.375
1,cargo_2035_23956ba0,2035-08-16,18.125
2,cargo_2035_23957cfd,2035-08-20,20.625
3,cargo_2035_23958501,2035-11-07,13.125
4,cargo_2035_23959ab6,2035-08-24,13.125
...,...,...,...
4987,cargo_2035_31187527,2035-08-21,7.500
4988,cargo_2035_3119010f,2035-08-24,10.000
4989,cargo_2035_3119118e,2035-08-25,15.000
4990,cargo_2035_311936c4,2035-08-30,19.000


In [71]:
trans

,source,target_harbor,Activities_harbor,fish_species_harbor,kind_harbor,fish_id,date,qty_tons
0,cargo_2035_2394778c,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,gadusnspecificatae4ba,2035-11-03,24.375
1,cargo_2035_23956ba0,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,gadusnspecificatae4ba,2035-08-16,18.125
2,cargo_2035_23957cfd,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,gadusnspecificatae4ba,2035-08-20,20.625
3,cargo_2035_23958501,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,gadusnspecificatae4ba,2035-11-07,13.125
4,cargo_2035_23959ab6,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,gadusnspecificatae4ba,2035-08-24,13.125
...,...,...,...,...,...,...,...,...
4987,cargo_2035_31187527,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,habeaspisces4eb,2035-08-21,7.500
4988,cargo_2035_3119010f,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,habeaspisces4eb,2035-08-24,10.000
4989,cargo_2035_3119118e,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,habeaspisces4eb,2035-08-25,15.000
4990,cargo_2035_311936c4,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,habeaspisces4eb,2035-08-30,19.000


In [72]:
selected_city = "City of Paackland"

# --- 1️⃣ FISH → LOCATION ---
links_fish_loc = (
    fish_locations.groupby(['entity_name', 'location_id'])
    .size()
    .reset_index(name='value')
    .rename(columns={'entity_name': 'Fish', 'location_id': 'Location'})
)

# --- 2️⃣ LOCATION → VESSEL ---
links_loc_vessel = (
    ping[['source', 'target']]
    .dropna()
    .rename(columns={'source': 'Location', 'target': 'Vessel'})
    .groupby(['Location', 'Vessel'])
    .size()
    .reset_index(name='value')
)
links_loc_vessel = links_loc_vessel[
    links_loc_vessel['Location'].isin(fish_locations['location_id'].unique())
]

# --- 3️⃣ VESSEL → CITY ---
vessel_city = ping[ping['source'].isin(cities['location_id'])].copy()
links_vessel_city = (
    vessel_city[['source', 'target']]
    .dropna()
    .rename(columns={'source': 'City', 'target': 'Vessel'})
    .groupby(['Vessel', 'City'])
    .size()
    .reset_index(name='value')
)

# --- 4️⃣ FISH → CITY (dalla tabella trans) ---
links_fish_city = (
    trans[['target_harbor', 'fish_id']]
    .dropna()
    .rename(columns={'target_harbor': 'City', 'fish_id': 'Fish'})
    .groupby(['Fish', 'City'])
    .size()
    .reset_index(name='value')
)

# --- 🔒 FILTRO: considera solo i flussi che arrivano alla city scelta ---
vessels_to_city = set(links_vessel_city.loc[links_vessel_city['City'] == selected_city, 'Vessel'])
fish_to_city = set(links_fish_city.loc[links_fish_city['City'] == selected_city, 'Fish'])

# Filtra Location → Vessel (solo se il vessel arriva alla city)
links_loc_vessel_f = links_loc_vessel[links_loc_vessel['Vessel'].isin(vessels_to_city)]
locations_kept = set(links_loc_vessel_f['Location'])

# Filtra Fish → Location (solo per location coinvolte)
links_fish_loc_f = links_fish_loc[links_fish_loc['Location'].isin(locations_kept)]

# Filtra VESSEL → CITY
links_vessel_city_f = links_vessel_city[links_vessel_city['City'] == selected_city]

# Filtra FISH → CITY
links_fish_city_f = links_fish_city[links_fish_city['City'] == selected_city]

# --- 5️⃣ Combina tutti i livelli ---
links_all_filtered = pd.concat([
    links_fish_loc_f.rename(columns={'Fish': 'source_name', 'Location': 'target_name'}),
    links_loc_vessel_f.rename(columns={'Location': 'source_name', 'Vessel': 'target_name'}),
    links_vessel_city_f.rename(columns={'Vessel': 'source_name', 'City': 'target_name'}),
    links_fish_city_f.rename(columns={'Fish': 'source_name', 'City': 'target_name'})  # 🔥 aggiunge collegamento diretto
], ignore_index=True)

# --- 6️⃣ Nodi e mapping ---
nodes = pd.Index(pd.concat([links_all_filtered['source_name'], links_all_filtered['target_name']]).dropna().unique())
links_all_filtered['source_id'] = links_all_filtered['source_name'].apply(nodes.get_loc)
links_all_filtered['target_id'] = links_all_filtered['target_name'].apply(nodes.get_loc)

# --- 7️⃣ Sankey ---
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    arrangement="snap",
    node=dict(
        label=nodes.tolist(),
        pad=20,
        thickness=18,
        line=dict(color="black", width=0.5)
    ),
    link=dict(
        source=links_all_filtered['source_id'],
        target=links_all_filtered['target_id'],
        value=links_all_filtered['value']
    )
)])

fig.update_layout(
    title_text=f"Fish → Location → Vessel → {selected_city} (+ direct Fish → City)",
    font_size=10,
    height=800
)

fig.show()


In [73]:
selected_city = "City of Paackland"

# --- 1️⃣ FISH → LOCATION ---
links_fish_loc = (
    fish_locations.groupby(['entity_name', 'location_id'])
    .size()
    .reset_index(name='value')
    .rename(columns={'entity_name': 'Fish', 'location_id': 'Location'})
)

# --- 2️⃣ LOCATION → VESSEL ---
links_loc_vessel = (
    ping[['source', 'target']]
    .dropna()
    .rename(columns={'source': 'Location', 'target': 'Vessel'})
    .groupby(['Location', 'Vessel'])
    .size()
    .reset_index(name='value')
)
links_loc_vessel = links_loc_vessel[
    links_loc_vessel['Location'].isin(fish_locations['location_id'].unique())
]

# --- 3️⃣ VESSEL → CITY ---
vessel_city = ping[ping['source'].isin(cities['location_id'])].copy()
links_vessel_city = (
    vessel_city[['source', 'target']]
    .dropna()
    .rename(columns={'source': 'City', 'target': 'Vessel'})
    .groupby(['Vessel', 'City'])
    .size()
    .reset_index(name='value')
)

# --- 4️⃣ CITY → FISH (dal dataset trans) ---
links_city_fish = (
    trans[['target_harbor', 'fish_id']]
    .dropna()
    .rename(columns={'target_harbor': 'City', 'fish_id': 'FishCommodity'})
    .groupby(['City', 'FishCommodity'])
    .size()
    .reset_index(name='value')
)

# --- 🔒 FILTRO: considera solo i flussi legati alla city selezionata ---
vessels_to_city = set(
    links_vessel_city.loc[links_vessel_city['City'] == selected_city, 'Vessel']
)
links_loc_vessel_f = links_loc_vessel[links_loc_vessel['Vessel'].isin(vessels_to_city)]
locations_kept = set(links_loc_vessel_f['Location'])
links_fish_loc_f = links_fish_loc[links_fish_loc['Location'].isin(locations_kept)]
links_vessel_city_f = links_vessel_city[links_vessel_city['City'] == selected_city]
links_city_fish_f = links_city_fish[links_city_fish['City'] == selected_city]

# --- 5️⃣ Combina tutti i livelli ---
links_all_filtered = pd.concat([
    links_fish_loc_f.rename(columns={'Fish': 'source_name', 'Location': 'target_name'}),
    links_loc_vessel_f.rename(columns={'Location': 'source_name', 'Vessel': 'target_name'}),
    links_vessel_city_f.rename(columns={'Vessel': 'source_name', 'City': 'target_name'}),
    links_city_fish_f.rename(columns={'City': 'source_name', 'FishCommodity': 'target_name'})  # ✅ estensione
], ignore_index=True)

# --- 6️⃣ Crea nodi e mapping ---
nodes = pd.Index(pd.concat(
    [links_all_filtered['source_name'], links_all_filtered['target_name']]
).dropna().unique())

links_all_filtered['source_id'] = links_all_filtered['source_name'].apply(nodes.get_loc)
links_all_filtered['target_id'] = links_all_filtered['target_name'].apply(nodes.get_loc)

# --- 7️⃣ Sankey finale ---
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    arrangement="snap",
    node=dict(
        label=nodes.tolist(),
        pad=20,
        thickness=18,
        line=dict(color="black", width=0.5)
    ),
    link=dict(
        source=links_all_filtered['source_id'],
        target=links_all_filtered['target_id'],
        value=links_all_filtered['value']
    )
)])

fig.update_layout(
    title_text=f"Fish → Location → Vessel → {selected_city} → Exported Fish",
    font_size=10,
    height=900
)

fig.show()


In [74]:
selected_port = "City of Paackland"
selected_date = pd.Timestamp("2035-11-23")

In [75]:
trans

,source,target_harbor,Activities_harbor,fish_species_harbor,kind_harbor,fish_id,date,qty_tons
0,cargo_2035_2394778c,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,gadusnspecificatae4ba,2035-11-03,24.375
1,cargo_2035_23956ba0,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,gadusnspecificatae4ba,2035-08-16,18.125
2,cargo_2035_23957cfd,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,gadusnspecificatae4ba,2035-08-20,20.625
3,cargo_2035_23958501,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,gadusnspecificatae4ba,2035-11-07,13.125
4,cargo_2035_23959ab6,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,gadusnspecificatae4ba,2035-08-24,13.125
...,...,...,...,...,...,...,...,...
4987,cargo_2035_31187527,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,habeaspisces4eb,2035-08-21,7.500
4988,cargo_2035_3119010f,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,habeaspisces4eb,2035-08-24,10.000
4989,cargo_2035_3119118e,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,habeaspisces4eb,2035-08-25,15.000
4990,cargo_2035_311936c4,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,habeaspisces4eb,2035-08-30,19.000


In [76]:
trans

,source,target_harbor,Activities_harbor,fish_species_harbor,kind_harbor,fish_id,date,qty_tons
0,cargo_2035_2394778c,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,gadusnspecificatae4ba,2035-11-03,24.375
1,cargo_2035_23956ba0,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,gadusnspecificatae4ba,2035-08-16,18.125
2,cargo_2035_23957cfd,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,gadusnspecificatae4ba,2035-08-20,20.625
3,cargo_2035_23958501,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,gadusnspecificatae4ba,2035-11-07,13.125
4,cargo_2035_23959ab6,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,gadusnspecificatae4ba,2035-08-24,13.125
...,...,...,...,...,...,...,...,...
4987,cargo_2035_31187527,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,habeaspisces4eb,2035-08-21,7.500
4988,cargo_2035_3119010f,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,habeaspisces4eb,2035-08-24,10.000
4989,cargo_2035_3119118e,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,habeaspisces4eb,2035-08-25,15.000
4990,cargo_2035_311936c4,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,habeaspisces4eb,2035-08-30,19.000


In [77]:
trans

,source,target_harbor,Activities_harbor,fish_species_harbor,kind_harbor,fish_id,date,qty_tons
0,cargo_2035_2394778c,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,gadusnspecificatae4ba,2035-11-03,24.375
1,cargo_2035_23956ba0,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,gadusnspecificatae4ba,2035-08-16,18.125
2,cargo_2035_23957cfd,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,gadusnspecificatae4ba,2035-08-20,20.625
3,cargo_2035_23958501,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,gadusnspecificatae4ba,2035-11-07,13.125
4,cargo_2035_23959ab6,City of South Paackland,"Industry, Fishing industry, Local shipping",Unknown,city,gadusnspecificatae4ba,2035-08-24,13.125
...,...,...,...,...,...,...,...,...
4987,cargo_2035_31187527,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,habeaspisces4eb,2035-08-21,7.500
4988,cargo_2035_3119010f,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,habeaspisces4eb,2035-08-24,10.000
4989,cargo_2035_3119118e,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,habeaspisces4eb,2035-08-25,15.000
4990,cargo_2035_311936c4,City of Paackland,"Industry, Fishing industry, Local shipping, To...",Unknown,city,habeaspisces4eb,2035-08-30,19.000


In [78]:
# Calcolo esportazioni totali per fish_id nel porto e data selezionati
daily_exports = (
    trans[(trans["target_harbor"] == selected_port) &
          (pd.to_datetime(trans["date"]).dt.date == selected_date.date())]
    .groupby("fish_id", as_index=False)["qty_tons"]
    .sum()
    .rename(columns={"qty_tons": "exports_tons"})
)

In [79]:
daily_exports

,fish_id,exports_tons
0,gadusnspecificatae4ba,44.250
1,habeaspisces4eb,48.250
2,labridaenrefert9be,49.250
3,piscisosseusb6d,140.375
4,piscissapidum9b7,46.250


In [80]:
# Le navi arrivate il giorno PRIMA sono quelle che hanno scaricato il pesce
arrival_date = selected_date - pd.Timedelta(days=1)

arrived_vessels = (
    ping[(ping["source"] == selected_port) &
         (ping["time"].dt.date == arrival_date.date())]
    [["target", "time"]]
    .drop_duplicates("target")
)

In [81]:
arrived_vessels

,target,time
9162,wahoowrangler016,2035-11-22 18:16:55.524485
9406,squidsquad7fd,2035-11-22 22:37:46.948070
9762,marlinmaster8ab,2035-11-22 10:09:43.018553
10263,breambanditc85,2035-11-22 06:37:39.830201
10429,neptunesnete8c,2035-11-22 05:37:27.126286
10482,mahimahimaster04f,2035-11-22 06:36:25.138144
11115,soleseeker47a,2035-11-22 00:40:34.637757
11276,baitedbreath538,2035-11-22 02:05:11.835091
11427,speckledtroutsaboteur509,2035-11-22 02:12:00.575186
11519,costasmeraldaac7,2035-11-22 17:39:13.588831


In [82]:
ping

,source,target,time,dwell,week,company,flag_country,length_overall,tonnage,Activities,fish_species_present,kind
0,City of Haacklee,perchplundererbc0,2035-09-16 04:06:48.185987,115074.790577,37,"Bennett, Jones and Miller",Oceanus,70.0,600.0,"Tourism, Local shipping",Unknown,city
1,City of Haacklee,perchplundererbc0,2035-09-20 05:21:33.678120,412706.321880,38,"Bennett, Jones and Miller",Oceanus,70.0,600.0,"Tourism, Local shipping",Unknown,city
2,City of Haacklee,perchplundererbc0,2035-09-28 04:31:47.118191,286092.881809,39,"Bennett, Jones and Miller",Oceanus,70.0,600.0,"Tourism, Local shipping",Unknown,city
3,City of Haacklee,perchplundererbc0,2035-10-04 04:59:36.052683,327623.947317,40,"Bennett, Jones and Miller",Oceanus,70.0,600.0,"Tourism, Local shipping",Unknown,city
4,City of Haacklee,perchplundererbc0,2035-10-15 04:26:14.647650,243225.352350,42,"Bennett, Jones and Miller",Oceanus,70.0,600.0,"Tourism, Local shipping",Unknown,city
...,...,...,...,...,...,...,...,...,...,...,...,...
218472,Tuna Shelf,maritimemajesticeb7,2035-11-09 16:10:38.373929,3443.278261,45,Unknown,Merigrad,90.0,3800.0,"Commercial fishing, Sport fishing","Tuna/Thunnini n.vera, Birdseye/Pisces frigus, ...",Fishing Ground
218473,Tuna Shelf,vesselvanguardbf2,2035-07-31 13:26:04.610993,7236.802726,31,Unknown,Orvietola,90.0,2100.0,"Commercial fishing, Sport fishing","Tuna/Thunnini n.vera, Birdseye/Pisces frigus, ...",Fishing Ground
218474,Tuna Shelf,vesselvanguardbf2,2035-09-20 23:27:08.743682,6776.912594,38,Unknown,Orvietola,90.0,2100.0,"Commercial fishing, Sport fishing","Tuna/Thunnini n.vera, Birdseye/Pisces frigus, ...",Fishing Ground
218475,Tuna Shelf,seawaysavvy102,2035-08-24 11:43:28.130654,7722.914478,34,Unknown,Uzifrica,190.0,36800.0,"Commercial fishing, Sport fishing","Tuna/Thunnini n.vera, Birdseye/Pisces frigus, ...",Fishing Ground


In [83]:
def compute_fishing_window_strict(ping):
    """
    Costruisce cicli di pesca tra due 'city' consecutive, indipendentemente dal nome del porto.
    """
    ping = ping.sort_values(["target", "time"])
    records = []

    for vessel, group in ping.groupby("target"):
        group = group.reset_index(drop=True)

        start_idx = None
        start_time = None

        for i, row in group.iterrows():
            # Inizio ciclo (una city)
            if row["kind"] == "city":
                # Se c'è già un ciclo aperto, chiudilo
                if start_idx is not None and i > start_idx + 1:
                    trip = group.loc[start_idx + 1 : i - 1]
                    if not trip.empty:
                        dwell_summary = (
                            trip.groupby(["source", "kind"], as_index=False)["dwell"].sum()
                        )
                        dwell_summary = dwell_summary.rename(columns={"source": "location_name"})
                        dwell_summary["vessel_name"] = vessel
                        dwell_summary["arrival_time"] = row["time"]
                        records.append(dwell_summary)

                # Nuovo ciclo (apertura)
                start_idx = i
                start_time = row["time"]

    return pd.concat(records, ignore_index=True)

In [84]:
df_fishing_window = compute_fishing_window_strict(ping)

In [88]:
df_fishing_window[df_fishing_window["vessel_name"]=="browntroutbandite67"]

,location_name,kind,dwell,vessel_name,arrival_time
7901,Cod Table,Fishing Ground,91657.447552,browntroutbandite67,2035-10-05 23:25:10.181513
7902,Exit East,buoy,12727.202670,browntroutbandite67,2035-10-05 23:25:10.181513
7903,Nav 1,buoy,18015.769023,browntroutbandite67,2035-10-05 23:25:10.181513
7904,Nav 2,buoy,17034.446207,browntroutbandite67,2035-10-05 23:25:10.181513
7905,Nav C,buoy,8598.138316,browntroutbandite67,2035-10-05 23:25:10.181513
7906,Cod Table,Fishing Ground,91656.098795,browntroutbandite67,2035-10-09 22:16:42.269553
7907,Exit East,buoy,12650.618453,browntroutbandite67,2035-10-09 22:16:42.269553
7908,Nav 1,buoy,18401.774342,browntroutbandite67,2035-10-09 22:16:42.269553
7909,Nav 2,buoy,15028.612525,browntroutbandite67,2035-10-09 22:16:42.269553
7910,Nav C,buoy,8008.553375,browntroutbandite67,2035-10-09 22:16:42.269553


In [89]:
# --- Uso pratico nello script principale ---
selected_port = "City of Paackland"
selected_date = pd.Timestamp("2035-11-23")

# 1️⃣ Calcolo routine di pesca dinamica
dwell_dynamic = compute_fishing_window_strict(ping)

# 2️⃣ Seleziono solo le navi che arrivano il giorno prima dell’export
arrival_date = selected_date - pd.Timedelta(days=1)
dwell_dynamic = dwell_dynamic[
    dwell_dynamic["arrival_time"].dt.date == arrival_date.date()
]

# 3️⃣ Aggiungo stazza (tonnage)
vessel_info = vessels[["vessel_id", "tonnage"]].rename(columns={"vessel_id": "vessel_name"})
dwell_dynamic = dwell_dynamic.merge(vessel_info,left_on="vessel_name", right_on="vessel_name", how="left")

# Calcolo esportazioni totali per fish_id nel porto e data selezionati
daily_exports = (
    trans[(trans["target_harbor"] == selected_port) &
          (pd.to_datetime(trans["date"]).dt.date == selected_date.date())]
    .groupby("fish_id", as_index=False)["qty_tons"]
    .sum()
    .rename(columns={"qty_tons": "exports_tons"})
)


# enrich con nomi pesci
id_to_name = dict(zip(fish["id"], fish["entity_name"]))
daily_exports["fish_name"] = daily_exports["fish_id"].map(id_to_name)

# 4️⃣ Preparo output per Step 2 / Step 3
daily_view = {
    "date": selected_date,
    "port": selected_port,
    "exports": daily_exports,        # dal blocco precedente
    "vessels_dwell": dwell_dynamic,  # finestra dinamica corretta
}

print("\n✅ Daily Harbor View (dynamic window) ready!")




✅ Daily Harbor View (dynamic window) ready!


In [90]:
def estimate_vessel_catch_by_habitat(daily_exports, dwell_dynamic, fish_locations):
    """
    Associa le esportazioni giornaliere alle navi candidate in base alle aree di pesca (habitat),
    pesando per tonnage × dwell solo nelle location associate al pesce.
    """
    results = []

    if daily_exports.empty or dwell_dynamic.empty:
        return pd.DataFrame(columns=[
            "vessel_name", "fish_name", "location_name", "dwell", "tonnage", "estimated_tons"
        ])

    # Crea una mappa fish → lista di location dove vive
    fish_to_locations = (
        fish_locations.groupby("entity_name")["location_id"]
        .apply(list)
        .to_dict()
    )

    for _, row in daily_exports.iterrows():
        fish_name = row["fish_name"]
        exports_tons = row["exports_tons"]

        habitats = fish_to_locations.get(fish_name, [])
        if not habitats:
            continue

        # Filtra solo dwell in location compatibili
        candidates = dwell_dynamic[dwell_dynamic["location_name"].isin(habitats)].copy()
        if candidates.empty:
            continue

        candidates["weight"] = candidates["tonnage"] * candidates["dwell"]
        total_weight = candidates["weight"].sum()
        if total_weight == 0:
            continue

        candidates["estimated_tons"] = exports_tons * (candidates["weight"] / total_weight)
        candidates["fish_name"] = fish_name

        results.append(
            candidates[["vessel_name", "fish_name", "location_name", "dwell", "tonnage", "estimated_tons"]]
        )

    if results:
        return pd.concat(results, ignore_index=True)
    else:
        return pd.DataFrame(columns=[
            "vessel_name", "fish_name", "location_name", "dwell", "tonnage", "estimated_tons"
        ])


In [91]:
vessel_catch = estimate_vessel_catch_by_habitat(
    daily_view["exports"],       # esportazioni del giorno
    daily_view["vessels_dwell"], # dwell calcolato con dynamic window
    fish_locations               # mappa pesce → habitat
)


In [94]:
vessel_catch

,vessel_name,fish_name,location_name,dwell,tonnage,estimated_tons
0,amberjackassaulterd52,Cod/Gadus n.specificatae,Cod Table,231544.984626,300.0,0.227012
1,baitedbreath538,Cod/Gadus n.specificatae,Cod Table,40291.877638,700.0,0.092174
2,barracudabaiter8b3,Cod/Gadus n.specificatae,Cod Table,159058.810187,100.0,0.051982
3,bassbaiterb9f,Cod/Gadus n.specificatae,Cod Table,37178.366872,1100.0,0.133652
4,bassbandit0d5,Cod/Gadus n.specificatae,Cod Table,135229.800800,600.0,0.265165
...,...,...,...,...,...,...
146,marlinmaster8ab,Harland/Piscis sapidum,Tuna Shelf,117106.230657,7800.0,18.928413
147,perchplundererbc0,Harland/Piscis sapidum,Tuna Shelf,117017.324789,600.0,1.454926
148,tunatrawlerafd,Harland/Piscis sapidum,Tuna Shelf,147419.236635,300.0,0.916463
149,welscatfishwrangler6ae,Harland/Piscis sapidum,Tuna Shelf,51689.288784,10800.0,11.568150


In [96]:
vessel_catch.sort_values("estimated_tons", ascending=False)

,vessel_name,fish_name,location_name,dwell,tonnage,estimated_tons
140,malta8cc,Offidiaa/Piscis osseus,Ghoti Preserve,297164.167144,200.0,140.375000
146,marlinmaster8ab,Harland/Piscis sapidum,Tuna Shelf,117106.230657,7800.0,18.928413
149,welscatfishwrangler6ae,Harland/Piscis sapidum,Tuna Shelf,51689.288784,10800.0,11.568150
124,marinemarauder8c9,Wrasse/Labridae n.refert,Wrasse Beds,63000.549259,17200.0,9.068903
15,grasspickerelgangster7d1,Cod/Gadus n.specificatae,Cod Table,143536.597566,11800.0,5.535241
...,...,...,...,...,...,...
67,kingfisher87d,Beauvoir/Habeas pisces,Cod Table,12652.406748,400.0,0.011276
14,cutthroattroutcatchere2b,Cod/Gadus n.specificatae,Cod Table,32516.388558,100.0,0.010627
59,costasmeraldaac7,Beauvoir/Habeas pisces,Cod Table,7810.522978,600.0,0.010441
60,cutthroattroutcatchere2b,Beauvoir/Habeas pisces,Cod Table,32516.388558,100.0,0.007244


In [87]:


# Seleziona una singola nave da visualizzare
selected_vessel = "browntroutbandite67"

df_vessel = ping[ping["target"] == selected_vessel].copy()

# Ordina cronologicamente e crea intervallo start-end per ciascun punto
df_vessel = df_vessel.sort_values("time")
df_vessel["start"] = df_vessel["time"]
df_vessel["end"] = df_vessel["time"] + pd.to_timedelta(df_vessel["dwell"], unit="s")

# Semplifica: tieni solo i campi rilevanti
df_vessel = df_vessel[["source", "kind", "start", "end"]]

# Colori diversi per tipo di area (porto, fishing ground, preserve, buoy)
color_map = {
    "city": "#1f77b4",
    "Fishing Ground": "#2ca02c",
    "Ecological Preserve": "#ff7f0e",
    "buoy": "#9467bd"
}

fig = px.timeline(
    df_vessel,
    x_start="start",
    x_end="end",
    y="source",
    color="kind",
    color_discrete_map=color_map,
    title=f"Fishing Routine – {selected_vessel}",
    labels={"location_name": "Location", "kind": "Type"}
)

# Inverti l’asse Y per avere i porti in alto
fig.update_yaxes(autorange="reversed")

# Miglioramenti estetici
fig.update_layout(
    height=600,
    xaxis_title="Time",
    yaxis_title="Locations Visited",
    legend_title="Location Type",
    template="simple_white"
)

fig.show()
